In [ ]:
!python --version """make sure its 3.10.x"""

In [ ]:
"""installing pip packages"""
!python -m pip install tf_slim
!python -m pip install pycocotools

In [ ]:
"""more pip packages"""

!python -m pip install Cython
!python -m pip install contextlib2
!python -m pip install pillow
!python -m pip install lxml


In [ ]:
"""being imports"""

import os

"""lets list out the images we got"""

for dirname, _, filenames in os.walk('./dataset/Images/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip show protobuf
# !protoc --version

In [ ]:
# """lets get some extra training data"""

!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
# python -m pip install --upgrade protobuf
cd models/research/
protoc --version
protoc object_detection/protos/*.proto --python_out=.

cp object_detection/packages/tf2/setup.py .
# pip install --upgrade setuptools
# pip install ez_setup
python -m pip install .

In [10]:
""" some imports, we need them for the next cell"""
""" if you got error running this. try updating ur protobuf version
    
"""
#

# !bash -c 'pip install protobuf==3.20.*'
!pip install --upgrade protobuf
import tensorflow as tf
# import io
# import numpy as np
# from PIL import Image
# from object_detection.utils import label_map_util
# from object_detection.utils import dataset_util
# from google.protobuf import text_format
# from object_detection.protos import string_int_label_map_pb2


Defaulting to user installation because normal site-packages is not writeable


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
def create_tf_example(example):

    prefix = "./dataset/Images/"
    filename = example["filename"]
    # Filename of the image. Empty if image is not from file

    with tf.io.gfile.GFile(filename, "rb") as fid:

        encoded_jpg = io.BytesIO()
        # encoded_jpg = io.BytesIO(fid.read(filename["filename"]))
        image = Image.open(f"./dataset/{filename}")
        image = image.resize((640, 640))

        image.save(encoded_jpg, format="JPEG")
        encoded_jpg = encoded_jpg.getvalue()

    width, height = image.size
    #     # TODO(user): Populate the following variables from your example.
    

    image_format = b"jpg"  # or b'png'

    xmins = example["xmin"]
    xmaxs = example["xmax"]

    ymins = example["ymin"]  #
    ymaxs = example["ymax"]  # List of normalized bottom y coordinates in bounding box

    classes_text = [
        "IssueBank",
        "ReceiverName",
        "AcNo",
        "Amt",
        "ChqNo",
        "DateIss",
        "Sign",
    ]  # List of string class name of bounding box (1 per box)
    classes_text = [x.encode("utf-8") for x in classes_text]
    classes = example["classid"].astype(
        np.int32
    )  # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "image/height": dataset_util.int64_feature(height),
                "image/width": dataset_util.int64_feature(width),
                "image/filename": dataset_util.bytes_feature(filename.encode()),
                "image/source_id": dataset_util.bytes_feature(filename.encode()),
                "image/encoded": dataset_util.bytes_feature(encoded_jpg),
                "image/format": dataset_util.bytes_feature(image_format),
                "image/object/bbox/xmin": dataset_util.float_list_feature(xmins),
                "image/object/bbox/xmax": dataset_util.float_list_feature(xmaxs),
                "image/object/bbox/ymin": dataset_util.float_list_feature(ymins),
                "image/object/bbox/ymax": dataset_util.float_list_feature(ymaxs),
                "image/object/class/text": dataset_util.bytes_list_feature(
                    classes_text
                ),
                "image/object/class/label": dataset_util.int64_list_feature(classes),
            }
        )
    )

    return tf_example


def get_img_info(filename):
    filename = filename.split("/")[-1]
    with open("./dataset/Images/" + filename.replace(".jpg", ".txt")) as f:
        image_annotations = f.readlines()
        f.close()
    for i in range(len(image_annotations)):
        image_annotations[i] = image_annotations[i].split(" ")
    image_annotations = np.array(image_annotations).astype(np.float)
    # image_annotations[:,1:] = np.array(image_annotations)[:,1:].astype(np.float)
    image_annotations[:, 0] = np.array(image_annotations)[:, 0].astype(np.int)
    return image_annotations


In [ ]:
writer = tf.io.TFRecordWriter("./train.record")

with open("./dataset/train.txt") as f:
    global files
    files = f.readlines()
    f.close()

for example in files:
    example = example.replace("\n", "")
    annotations = get_img_info(example)
    # print (example)
    
    ann_dict = {
        "filename": example,
        "xmin": annotations[:, 1],
        "ymin": annotations[:, 2],
        "xmax": annotations[:, 1] + annotations[:, 3],
        "ymax": annotations[:, 2] + annotations[:, 4],
        "classid": annotations[:, 0],
    }
    tf_example = create_tf_example(ann_dict)
    writer.write(tf_example.SerializeToString())

writer.close()


In [7]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2023-01-14 20:47:47--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.4.128, 2404:6800:4003:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.4.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90453990 (86M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v1_fp 100%[===================>]  86.26M  1006KB/s    in 74s     

2023-01-14 20:49:01 (1.16 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’ saved [90453990/90453990]

